## Updated Code for Implementing VAR Model

We are going to extract time series features and cluster series based on their feature values. We use the bisection k-means algorithm as it tends to produce clusters of similar size, and we can specify the number of clusters prior. E.g., we can pick a cluster number that leads to VAR models that are estimatable.

In [1]:
# load modules
import numpy as np
import pandas as pd
import pmdarima as pm
import statsmodels.api as sm
from sktime.transformations.series.difference import Differencer
from sktime.performance_metrics.forecasting import mean_absolute_error, MeanAbsoluteError
from sktime.forecasting.var import VAR
from data_protection_functions import *
from data_processing_functions import *
from forecasting_functions import *
# nice time series plots
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import ForecastingGridSearchCV, ExpandingWindowSplitter, SlidingWindowSplitter
from sktime.forecasting.compose import TransformedTargetForecaster

In [2]:
# import weekly finance time series
# ignore header and skip the first row to use integers as column names
full_data = pd.read_csv("../../Data/Train/Clean/full_m3_monthly_micro_clean.csv", header=None, skiprows=1)

In [3]:
# convert to a list of series, potentially with different lengths
full_data = [x.dropna() for _, x in full_data.iterrows()]

***

In [4]:
# forecast horizon
h = 1

In [5]:
Y = [x.iloc[:-h] for x in full_data]
Test = [x.iloc[-h:] for x in full_data]

In [6]:
Test = pd.DataFrame([x.reset_index(drop=True) for x in Test]).T

***

Apply clustering to each length of series.

Import the original time series features.

In [7]:
feats = pd.read_csv("../../Data/Train/Clean/tsfeatures/tsfeatures_h1.csv")

In [8]:
feats = feats.drop(["nperiods", "seasonal_period", "spike", "linearity", "curvature", "e_acf10",
                    "peak", "trough"], axis=1)

In [9]:
series_lengths = [len(y) for y in Y]

In [10]:
unique_lengths = np.unique(series_lengths)

In [76]:
length = unique_lengths[1]

In [77]:
tsd = [Y[i] for i,l in enumerate(series_lengths) if l == length]
tsi = [i for i,l in enumerate(series_lengths) if l == length]

In [78]:
f = feats.iloc[tsi]

In [79]:
from sklearn.preprocessing import StandardScaler

In [80]:
scaler = StandardScaler()

In [81]:
f_scaled = scaler.fit_transform(f)

In [82]:
from sklearn.cluster import BisectingKMeans

In [96]:
bisect_means = BisectingKMeans(n_clusters=int(np.ceil(len(tsd)/8)), n_init=20, bisecting_strategy="largest_cluster").fit(f_scaled)

C:\Users\Cameron\anaconda3\lib\site-packages\sklearn\cluster\_bisect_k_means.py:259: UserWarning: BisectingKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [97]:
cluster_ids, counts = np.unique(bisect_means.labels_, return_counts=True)

In [98]:
np.sum(counts==1)

1

So now we have the cluster assignments for the series. Apply differencing and forecast for each cluster.

In [210]:
ids = [i for i,j in enumerate(bisect_means.labels_) if j == cluster_ids[2]]

In [211]:
series = [tsd[i] for i,j in enumerate(bisect_means.labels_) if j == cluster_ids[2]]

***

## Step 2: Apply Data Protection to Generate Protected Series

In [212]:
Y_protected = apply_data_protection(Y, epsilon=20)

***

## Step 3: Pre-process the data.

In [213]:
Y_processed, _, _, _  = pre_process(ts_data=series, 
                                    target_forecast_period=h,
                                    log=True, 
                                    make_stationary=True)

# Y_protected_processed, Y_protected_last_window, Y_protected_last_window_trend, _, full_lags_protected = pre_process(ts_data=Y_protected, 
#                                                                                                target_forecast_period=h,
#                                                                                                log=True,  
#                                                                                                make_stationary=True, 
#                                                                                                sp=12)

***

In [214]:
# forecasts_original = post_process(full_ts_data=series, 
#                                   forecasts=Y_processed, 
#                                   target_forecast_period=h,
#                                   log=True,
#                                   make_stationary=True)

***

***

## Step 4: Train Models and Generate Forecasts

In [215]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [216]:
D = pd.concat(Y_processed, axis=1, ignore_index=True)

In [217]:
var_mod = VAR(D)

In [218]:
var_results = var_mod.fit(ic='bic', trend='c')

In [219]:
var_results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 12, Oct, 2022
Time:                     14:42:53
--------------------------------------------------------------------
No. of Equations:         11.0000    BIC:                   -24.2628
Nobs:                     66.0000    HQIC:                  -24.4835
Log likelihood:          -206.434    FPE:                2.01519e-11
AIC:                     -24.6277    Det(Omega_mle):     1.70796e-11
--------------------------------------------------------------------
Results for equation 0
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const        -0.010119         0.038810           -0.261           0.794

Results for equation 1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------

In [220]:
pd.concat([pd.DataFrame(var_results.coefs[:,:,i]) for i in range(var_results.coefs.shape[2])], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,1,2,3,4,5,6,7,8,9,10


In [174]:
intercepts = var_results.coefs_exog

In [175]:
lag_order = var_results.k_ar

In [176]:
if lag_order == 0:
    fcasts = np.repeat(intercepts, h, axis=1).T
else:
    fcasts = var_results.forecast(D.values[-lag_order:], h)

In [177]:
fcasts

array([[-0.01011863, -0.00757358, -0.01227335, -0.01454206, -0.00940646,
        -0.0135427 , -0.00664023, -0.01174647, -0.00719372, -0.00871082,
        -0.00682449]])

In [178]:
fcasts = [pd.Series(fcasts[:,i]) for i in range(len(Y_processed))]

In [180]:
fcast_indexes = [np.arange(series[i].index[-1]+1, series[i].index[-1]+h+1) for i in range(len(series))]

In [181]:
for i,f in enumerate(fcasts):
    f.index = fcast_indexes[i]

In [182]:
# forecasts_original = train_and_forecast(ts_data=Y_processed,
#                                         horizon_length=h,
#                                         forecasting_model="VAR")

# forecasts_protected = train_and_forecast(ts_data=Y_protected_processed,
#                                          horizon_length=h,
#                                          forecasting_model="VAR")

***

## Post Process the Forecasts

In [183]:
forecasts_original = post_process(full_ts_data=series, 
                                  forecasts=fcasts, 
                                  target_forecast_period=h,
                                  log=True,
                                  make_stationary=True)

# forecasts_protected = post_process(full_ts_data=Y_protected, 
#                                    forecasts=forecasts_protected, 
#                                    target_forecast_period=h,
#                                    log=True,
#                                    make_stationary=False,
#                                    sp=12,
#                                    full_lags=full_lags_protected)

In [184]:
forecasts_original

,0,1,2,3,4,5,6,7,8,9,10
0,2375.837746,3275.101575,2469.504157,1419.21096,2129.870926,1509.41935,1986.763543,2194.075406,3365.700786,2755.889074,2304.221081


In [185]:
Test.iloc[:,ids]

,8,9,14,15,16,56,155,169,179,184,186
0,4240.0,4100.0,1360.0,1180.0,1560.0,3650.0,4400.0,6650.0,2440.0,1320.0,2300.0


In [186]:
mean_absolute_error(Test.iloc[:,ids], forecasts_original)

1453.0181241237158

***